### Que compone el entrenamiento
- Labelmap
- Tfrecords (train, test)
- Modelo pre-entrenado
- Configurtar pipeline.config

In [ ]:
import os
import pandas as pd
import json
import pickle
import zipfile
import shutil

# Instalamos los paquetes necesarios para que funcione desde la Colab
!pip install avro-python3
!pip install tf_slim==1.1.0
!pip install tf-models-official
!pip install lvis
!pip install tensorflow_io
!pip install keras==2.7.0
!pip install opencv-python-headless

In [3]:
import os
%cd /content
!git clone --quiet https://github.com/tensorflow/models.git
%cd /content/models/
#!git checkout 58d19c67e1d30d905dd5c6e5092348658fed80af
!apt-get update && apt-get install -y -qq protobuf-compiler python-pil python-lxml python-tk
!pip install -q Cython contextlib2 pillow lxml matplotlib
!pip install -q pycocotools
%cd /content/models/research
!protoc object_detection/protos/*.proto --python_out=.
os.environ['PYTHONPATH'] += ':/content/models/research/:/content/models/research/slim/'
!python object_detection/builders/model_builder_test.py

[Errno 2] No such file or directory: '/content'
/Users/jucar/Documents/Projects/Curso computer vision/Notebooks
[Errno 2] No such file or directory: '/content/models/'
/Users/jucar/Documents/Projects/Curso computer vision/Notebooks
zsh:1: command not found: apt-get
[Errno 2] No such file or directory: '/content/models/research'
/Users/jucar/Documents/Projects/Curso computer vision/Notebooks
zsh:1: no matches found: object_detection/protos/*.proto
/Users/jucar/.local/share/virtualenvs/Curso_computer_vision-agCS5cFW/bin/python: can't open file '/Users/jucar/Documents/Projects/Curso computer vision/Notebooks/object_detection/builders/model_builder_test.py': [Errno 2] No such file or directory


In [3]:
!wget --no-check-certificate http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz \
    -O /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2022-04-29 01:19:28--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.148.128, 2607:f8b0:4001:c54::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.148.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’

/content/ssd_mobile 100%[===================>]  19.56M  --.-KB/s    in 0.07s   

2022-04-29 01:19:29 (266 MB/s) - ‘/content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz’ saved [20515344/20515344]



In [4]:
!tar -zxvf /content/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
output_path = 'ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8'
output_path = os.path.join(os.getcwd(), output_path)
print("La carpeta se almaceno en {}".format(output_path))

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/pipeline.config
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/saved_model.pb
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/saved_model/variables/variables.index
La carpeta se almaceno en /content/models/research/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8


In [5]:
path_training = '/content/ssd_mobilenet'
os.mkdir(path_training)

In [6]:
source_config = "{}/pipeline.config".format(output_path)
target_config = "{}/pipeline.config".format(path_training)
shutil.copyfile(source_config, target_config)

'/content/ssd_mobilenet/pipeline.config'

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
config = config_util.get_configs_from_pipeline_file(target_config)

In [9]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(target_config, "r") as f:
  proto_str = f.read()
  text_format.Merge(proto_str, pipeline_config)

In [10]:
# Definimos las rutas en donde se encuentra los TFRecords y el label map para agregarlos al archivo de configuración del pipeline.config
label_map_pbtxt_fname = "/content/cars_label_map.pbtxt"
train_record_fname = "/content/cars_train.tfrecord"
test_record_fname = "/content/cars_test.tfrecord"

In [11]:
# Cantidad de clases del modelo
pipeline_config.model.ssd.num_classes = 2

# El tamaño del batch size, entre más grande mas costo computacional va a necesitar en el proceso de entrenamiento, pero a su vez entrenara más rapido.
pipeline_config.train_config.batch_size = 4

# Donde almacenaremos los resultados del entrenamiento
pipeline_config.train_config.fine_tune_checkpoint ="{}/checkpoint/ckpt-0".format(output_path)

# Qué tipo de deteción aplicaremos (Object detection)
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"

# Dirección del label map
pipeline_config.train_input_reader.label_map_path = label_map_pbtxt_fname

# Dirección del train TFRecord
pipeline_config.train_input_reader.tf_record_input_reader.input_path[0] = train_record_fname

# Dirección del label map
pipeline_config.eval_input_reader[0].label_map_path = label_map_pbtxt_fname

# Dirección del test TFRecord
pipeline_config.eval_input_reader[0].tf_record_input_reader.input_path[0] = test_record_fname

In [12]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(target_config, "wb") as f:
  f.write(config_text)

In [ ]:
num_steps = 5000
model_dir = "/content/ssd_mobilenet"

# Utilizamos directamente la libreria de object detection para ejecutar el script model_main_tf2
# Los argumentos principales son el archivo de configuración, la ubicación del modelo y la cantidad de steps que ejecutará.
!python /content/models/research/object_detection/model_main_tf2.py \
--pipeline_config_path={target_config} \
--model_dir={model_dir} \
--num_train_steps={num_steps}

2022-04-29 01:20:00.366758: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
I0429 01:20:00.408984 140422678472576 mirrored_strategy.py:374] Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Maybe overwriting train_steps: 5000
I0429 01:20:00.413682 140422678472576 config_util.py:552] Maybe overwriting train_steps: 5000
INFO:tensorflow:Maybe overwriting use_bfloat16: False
I0429 01:20:00.413877 140422678472576 config_util.py:552] Maybe overwriting use_bfloat16: False
Instructions for updating:
rename to distribute_datasets_from_function
W0429 01:20:00.587341 140422678472576 deprecation.py:343] From /content/models/research/object_detection/model_lib_v2.py:564: StrategyBase.experimental_distribute_

## Análisis de los resultados con TensorBoard

In [ ]:
%load_ext tensorboard
%tensorboard --logdir "/content/ssd_mobilenet"

## Exportación de modelo

Exportamos nuestro modelo para poder utilizarlo en cualquier otro momento.


In [ ]:
output_directory = "/content/fine_tuned_model"

!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path {target_config} \
--trained_checkpoint_dir {model_dir} \
--output_directory {output_directory}